# pyAFQ: Automated Fiber Quantification in Python 

The core of the tractometry.org ecosystem is the pyAFQ software library. This
library uses inputs from various other programs to perform the delineation of
major white matter pathways and to quantify white matter tissue properties
along their lengths - tract profiles. The final output of the program includes
a tabular summary with these tract profiles for each subject in the dataset,
and one merged table that includes the tract profiles for all of the subjects. 

Along the way, the program produces many different intermediate data, including 
tractography results, maps of tissue properties, and visualizations, which 
can be used for quality assurance of the data and processing.

There are two ways to use pyAFQ: through a command line interface, and by 
writing Python code. Here, we will focus on the latter, using pyAFQ's 
Python Application Programming Interface (API).

## Setup 

We start with a few setup steps. The pyAFQ software downloads templates that 
are required for anatomical delineation of tracts into a designated directory 
on the user's machine. For the purpose of these examples, we will set the 
path to this directory to `~/data/tractometry`. This is done in code that is 
executed when we import the code in the `paths.py` file. You can open that file 
and take a look if you are curious what this does. Once that code is executed, 
we will have a pointer to this directory in the `afq_home` variable, which 
we will also use to point to datasets. You will see a similar import at the top 
of every notebook here, and it always does the same thing.

In [ ]:
import os.path as op
from paths import afq_home

Next, we import the `GroupAFQ` class from the pyAFQ `api` module. We also 
import `plotly`, which we will use for visualization.

In [ ]:
from AFQ.api.group import GroupAFQ
import plotly

## Set up tracking parameters 

Before we initialize the `GroupAFQ` object, we create a `tracking_params`
variable, which we will pass to the `GroupAFQ` object to specify the
tractography part of the pipeline. Here, we specify that we want 100,000 seeds
randomly distributed in the white matter. We also set ``num_chunks`` to `True`.
This will tell pyAFQ to use the `ray` software library to parallelize the
tracking across all cores. This can be removed to process in serial, or set to
use a particular distribution of work by setting `num_chunks` to an integer
number.



In [ ]:
tracking_params = dict(n_seeds=100000,
                       random_seeds=True,
                       rng_seed=2022,
                       trx=True,
                       num_chunks=True)

## Initialize an AFQ object

Next, we initialize the `GroupAFQ` object. Upon initialization, the object 
does some validation steps, but no major computation is triggered. Instead, 
computation is deferred until the `export` method is called.

In [ ]:
myafq = GroupAFQ(
    bids_path=op.join(afq_home, 'stanford_hardi'),
    preproc_pipeline='vistasoft',
    tracking_params=tracking_params,
    viz_backend_spec='plotly_no_gif')

## Recognize the tracts and calculating tract profiles:

The `myafq` class instance now has all the information that it needs in order to 
do all kinds of computations on the data. Typically, we will want to calculate 
the tract profiles. To trigger the
pyAFQ pipeline that calculates the profiles, users can call the
`export('profiles')` method.

```{alert}

For the purpose of this demonstration, the pipeline has been run in advance and
cached in the data folder (under `data/tractometry`). In usual practice, the
following line of code would trigger the full computational pipeline, which
takes about 10 minutes to run and requires about 6GB RAM (more than is
available through the neurolibre server). However, the caching behavior is
typical to pyAFQ. Once derivatives are computed, they are cached such that they
can be reused. 
```


In [ ]:
myafq.export('profiles')

After this line of code has finished running, a CSV file that contains the 
tract profiles should be saved into the afq folder within the derivatives 
folder of the dataset. At this point, we can export a visualization that 
shows all of the tracts and tract profiles of FA by calling `export` again. 
The call to plotly then renders the html of the figure into the notebook.

In [ ]:
bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

Tract profile data for all subjects in a particular dataset can be accessed as 
a Pandas dataframe, using the `combine_profiles` method.  

In [ ]:
profiles_df = myafq.combine_profiles()

# Visualize tract profiles 

With a few lines of [dash](https://dash.plotly.com/) Python code we can
construct a small interactive dashboard that displays the FA tract profiles of
each of the tracts:

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('DTI FA Profiles'),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=list(profiles_df.tractID.unique()),
        value=["Left Arcuate", "Right Arcuate"],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"),
    Input("checklist", "value"))
def update_line_chart(tracts):
    df = profiles_df
    mask = df.tractID.isin(tracts)
    fig = px.line(df[mask],
        x="nodeID", y="dti_fa", color='tractID')
    return fig


app.run_server(debug=True)